# Imports and setting up viz

NB : conda lam1env (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_hf import *
from tools_native import *
from tools_LIAISE import *
from tools_mesoNH import *

# sys.path.append('../../python_tools/aborella/PLOTS/')
# from datasets import *
sys.path.append('../../python_tools/aborella/UTIL/')
import xr_utils

In [ ]:
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [10., 8.]

# Load model files and edit datasets

In [ ]:
mesoNH_dir = '../../../mesoNH_simulations'

In [ ]:
def format_MesoNH_outputs_basic(filename, ds_name=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name']= ds_name

    rename_dict={
        'time':'start_time',
        'longitude':'lon',
        'latitude':'lat'
    }
    ds = ds.rename(rename_dict)

    # Add time coordinate
    # Assuming 'time' coordinate is the starting time.
    base_time = pd.to_datetime(ds["start_time"].isel(start_time=0).item())
    print(f"Base time for the dataset: {base_time}")
    # Create a timedelta for each hour
    hourly_offsets = pd.to_timedelta(ds["record"].values, unit='h')
    new_time_vals = base_time + hourly_offsets
    # Assign new_time as a new coordinate, linked to the 'record' dimension
    ds = ds.assign_coords(time=("record", new_time_vals))
    ds = ds.swap_dims({"record": "time"})
    #drop start_time and record
    ds = ds.drop_vars(['start_time'])

    if 'LE_ISBA' in ds:
        ds = ds.rename({'LE_ISBA': 'flat'})
        ds['flat'].attrs['long_name'] = 'Latent Heat Flux on natural areas'
        ds['flat'].attrs['units'] = 'W m⁻²'
    if 'H_ISBA' in ds:
        ds = ds.rename({'H_ISBA': 'sens'})
        ds['sens'].attrs['long_name'] = 'Sensible Heat Flux on natural areas'
        ds['sens'].attrs['units'] = 'W m⁻²'
    if 'HU2M_ISBA' in ds:
        ds['rh2m'] = ds['HU2M_ISBA'] * 100.0 
        ds['rh2m'].attrs['long_name'] = 'Relative Humidity at 2m'
        ds['rh2m'].attrs['units'] = '%'
    if 'T2M_ISBA' in ds:
        ds = ds.rename({'T2M_ISBA': 't2m'})
        ds['t2m'].attrs['long_name'] = 'Temperature at 2m'
        ds['t2m'] = ds['t2m'] - 273.15
        ds['t2m'].attrs['units'] = '°C'
    if 'SWD' in ds:
        ds = ds.rename({'SWD': 'SWdnSFC'})
        ds['SWdnSFC'].attrs['long_name'] = 'Downward Shortwave Radiation at Surface'
        ds['SWdnSFC'].attrs['units'] = 'W m⁻²'

    return(ds)

In [ ]:
filename='{}/LIAIS.1*.nc'.format(mesoNH_dir)
ds_orig = format_MesoNH_outputs_basic(filename)
ds_orig

# Maps

In [ ]:
def map_mesoNH_timestamp_restrict(ds, var, vmin=None, vmax=None, cmap='viridis',
                         add_liaise=False,
                         timestamp='2021-07-14T01:00:00',
                         lon_min=None, lon_max=None, lat_min=None, lat_max=None):
    # Select data for the given timestamp
    data_to_plot_time_selected = ds[var].sel(time=timestamp, method='nearest')

    # Get the 2D lon/lat coordinates from the dataset (they have nj, ni dimensions)
    lons_coord = ds['lon']
    lats_coord = ds['lat']

    # Apply spatial subsetting if bounds are provided
    if lon_min is not None and lon_max is not None and \
       lat_min is not None and lat_max is not None:

        # Create a boolean mask based on the 2D lon/lat coordinates
        # This mask will have dimensions (nj, ni)
        spatial_mask = (lons_coord >= lon_min) & (lons_coord <= lon_max) & \
                       (lats_coord >= lat_min) & (lats_coord <= lat_max)

        # Apply the mask to the data. Values outside the region will become NaN.
        data_to_plot = data_to_plot_time_selected.where(spatial_mask)
    else:
        # If no subsetting is requested, use the time-selected data directly
        data_to_plot = data_to_plot_time_selected
        
    title = f'{var} on {pd.to_datetime(timestamp).strftime("%Y-%m-%d %H:%M UTC")}'
    label = f'{var} ({ds[var].attrs.get("units", "")})'

    nice_map_mesoNH(data_to_plot, vmin=vmin, vmax=vmax, cmap=cmap,
                         add_liaise=add_liaise, title=title, label=label)


In [ ]:
var='rh2m'
vmin=0
vmax=100
# vmin,vmax=None, None
cmap=blues
map_mesoNH_timestamp_restrict(ds_orig, 
                     var,
                     vmin=vmin, vmax=vmax, cmap=cmap,
                     timestamp='2021-07-20T12:00:00',
                     add_liaise=False,
                    lon_min=-1, lon_max=1, lat_min=41, lat_max=42
                     )

In [ ]:
var='flat'
vmin=0
vmax=250
# vmin,vmax=None, None
cmap=blues
map_mesoNH_mean(ds_orig, 
                 var,
                 vmin=vmin, vmax=vmax, cmap=cmap,
                 title='Latent Heat Flux mean (14-31/07/2021)',
                 add_liaise=True
                 )

# Time series

In [ ]:
var='flat'
ds=ds_orig
ds_list=[ds]

time_series_ave(ds_list, var, ds_colors=False, title='{} {}'.format(var, ds[var].attrs['units']))
# seasonal_cycle_ave(ds_list, var, ds_colors=True, year_min=year_min, year_max=year_max, ylabel=ylabel, title=title, figsize=figsize)

# Compare with LMDZ

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import xesmf as xe

def reformat_datasets_for_comparison(ds_source_initial, ds_target_rectilinear, var_names):
    """
    Reformats two datasets for comparison:
    1. Converts the source dataset (with record, nj, ni dimensions) to have a 'time' dimension
       based on record index, and regrids it onto the target dataset's rectilinear grid.
    2. Aligns the time dimension of both datasets.

    Args:
        ds_source_initial (xr.Dataset): The source dataset with LE_ISBA(record, nj, ni)
                                        and 2D lat/lon coordinates (nj, ni).
                                        Must have 'time' coordinate (record, datetime64)
                                        and 'record' coordinate (record).
        ds_target_rectilinear (xr.Dataset): The target dataset with LE_ISBA(time, lat, lon)
                                            and 1D lat/lon dimensions.
        var_names (list): A list of variable names (strings) to reformat and compare.

    Returns:
        tuple: A tuple containing (ds_source_regridded_aligned, ds_target_aligned).
               Both datasets will have (time, lat, lon) dimensions and aligned times.
    """
    # Prepare ds_source_initial: add 'new_time' and swap 'record' dimension
    base_time = pd.to_datetime(ds_source_initial["time"].isel(record=0).item())
    hourly_offsets = pd.to_timedelta(np.arange(ds_source_initial.sizes["record"]), unit='H')
    new_time_values = base_time + hourly_offsets
    
    ds_source_prepared = ds_source_initial.assign_coords(new_time=("record", new_time_values))
    ds_source_prepared = ds_source_prepared.swap_dims({'record': 'new_time'})
    ds_source_prepared = ds_source_prepared.drop_vars('record')
    ds_source_prepared = ds_source_prepared.rename_dims({'new_time': 'time'}) # Ensure time dim name consistency

    # Initialize regridder for spatial transformation
    regridder = xe.Regridder(ds_source_prepared, ds_target_rectilinear, method='bilinear', periodic=False)

    # Regrid specified variables from ds_source
    regridded_vars = {}
    for var_name in var_names:
        regridded_vars[var_name] = regridder(ds_source_prepared[var_name])

    # Combine regridded variables into a new Dataset
    ds_source_regridded = xr.Dataset(regridded_vars, coords=regridded_vars[var_names[0]].coords)

    # Align time dimensions of both datasets for comparison
    # Use reindex_like to ensure common time steps; fill with NaN if no match
    ds_source_aligned = ds_source_regridded.reindex_like(ds_target_rectilinear, method='nearest', tolerance=pd.Timedelta('0.5 hours'))
    ds_target_aligned = ds_target_rectilinear.reindex_like(ds_source_regridded, method='nearest', tolerance=pd.Timedelta('0.5 hours'))
    
    return ds_source_aligned, ds_target_aligned